# Token Issuer Notebook


## Configure Token Information

This section enables customization of the token that will be minted. Edit the variables in the section below as necessary.

* identity : the identity of the token creator
* service_host : the host for the eservice where tokens will be minted, this will use the default service group
* token_class : the name of tokens that will be generated, this is only used to simplify local access (e.g. context file name)
* token_description : a description of the asset associated with the minted tokens
* token_metadata : additional information about the token
* count : the number of tokens to mint for the asset

Note that the notebook assumes that there is a key file for the identity of the form: `${keys}/${identity}_private.pem`.

In [ ]:
token_owner = 'user1'
token_class = 'mytoken'
token_description = 'this is my token'
token_metadata = 'created by {}'.format(token_owner)
count = 1
service_host = 'localhost'
notebook_directory = '../..'

<hr style="border:2px solid gray">

## Initialize

In [ ]:
import os
import pdo.inference.jupyter as ex_jupyter
import IPython.display as ip_display

ex_jupyter.load_ipython_extension(get_ipython())

### Initialize the PDO Environment

Initialize the PDO environment. This assumes that a functional PDO configuration is in place and that the PDO virtual environment has been activated. In particular, ensure that the groups file and eservice database have been configured correctly. If you do not have a service groups configuration, you can create it for a single service host by running the following:

In [ ]:
%%skip True
%%bash -s $service_host
if [ ! -f $PDO_HOME/etc/$1_groups.toml ] ; then 
    $PDO_INSTALL_ROOT/bin/pdo-shell $PDO_HOME/bin/pdo-create-service-groups.psh --service_host $1
fi

For the most part, no modifications should be require below.

In [ ]:
common_bindings = {
    'host' : service_host,
    'token_owner' : token_owner,
    'token_class' : token_class,
}

(state, bindings) = ex_jupyter.initialize_environment(token_owner, **common_bindings)
print('environment initialized')

### Initialize the Contract Context

The contract context defines the configuration for a collection of contract objects that interact with one another. By default, the context file used in this notebook is specific to the token. If you prefer to use a common context file, edit the `context_file` variable below.

For the most part, no other modifications should be required.

In [ ]:
token_path = 'token.' + token_class
context_file = bindings.expand('${etc}/${token_class}_context.toml')
print("using context file {}".format(context_file))

context_bindings = {
    'asset_type.identity' : token_owner,
    'vetting.identity' : token_owner,
    'guardian.identity' : token_owner,
    'token_issuer.identity' : token_owner,
    'token_issuer.count' : count,
    'token_issuer.description' : token_description,
    'token_issuer.token_metadata.opaque' : token_metadata,
    'token_object.identity' : token_owner,
    'guardian.url' : 'http://' + service_host + ':7900'
}

context = ex_jupyter.initialize_token_context(state, bindings, context_file, token_path, **context_bindings)
ex_jupyter.pbuilder.Context.SaveContextFile(state, context_file)
print('context initialized')

### Create the Token Issuer Contract

The process of creating the token issuer will also create an asset type contract object, a vetting organization contract object, and the guardian contract object. The asset type and vetting organization contract objects are principally used to complete the canonical asset interface that enables transparent value exchanges with tokens and other digital assets. 

Please ensure that guardian service webserver (both frontend and backend) is running prior to creating the token issuer contract. Please see inference-contract [documentation](../../../inference-contract//README.md) for details on how to deploy the guardian service.

In [ ]:
token_issuer_context = ex_jupyter.pbuilder.Context(state, token_path + '.token_issuer')
token_issuer_save_file = token_issuer_context.get('save_file')
if not token_issuer_save_file :
    token_issuer_save_file = ex_jupyter.pcommand.invoke_contract_cmd(
        ex_jupyter.ex_token_issuer.cmd_create_token_issuer, state, token_issuer_context)
    ex_jupyter.pbuilder.Context.SaveContextFile(state, context_file)
print('token issuer contract in {}'.format(token_issuer_save_file))

<hr style="border:2px solid gray">

## Operate on the Contract

### Mint the Tokens

In [ ]:
token_object_context = ex_jupyter.pbuilder.Context(state, token_path + '.token_object')

minted_token_save_files = ex_jupyter.pcommand.invoke_contract_cmd(
    ex_jupyter.inference_token_object.cmd_mint_tokens, state, token_object_context)
ex_jupyter.pbuilder.Context.SaveContextFile(state, context_file)

minted_token_contexts = []
for token_index in range(1, len(minted_token_save_files)+1) :
    minted_token_contexts += [ token_object_context.get_context('token_{}'.format(token_index)) ]
    
print("{} tokens minted".format(len(minted_token_save_files)))

### Create Token Notebooks

Create a token notebook for each of the minted tokens.

In [ ]:
%%skip True

import os
import papermill as pm

instance_directory = os.path.join(notebook_directory, 'instances', token_class)
template_file = os.path.join(notebook_directory, 'templates', 'token.ipynb')

os.makedirs(instance_directory, exist_ok=True)

instance_parameters = {
    'token_owner' : token_owner,
    'token_class' : token_class,
    'context_file' : context_file,
    'service_host' : service_host,
}

for token_context in minted_token_contexts :
    instance_parameters['token_path'] = token_context.path
    instance_parameters['token_name'] = token_context.path.split('.')[-1]
    instance_file = os.path.join(instance_directory, instance_parameters['token_name'] + '.ipynb')
    if not os.path.exists(instance_file) :
        pm.execute_notebook(
            template_file,
            instance_file,
            prepare_only=True,
            parameters=instance_parameters,
        )
    instance_path = os.path.basename(instance_file)
    ip_display.display(ip_display.Markdown('[{}]({})'.format(token_context.path, instance_path)))

<hr style="border:2px solid gray">

## Contract Metadata

### Contract Save Files

This notebook contains three contract files. Detailed information about the contracts can be found below.

In [ ]:
%%skip True
contract_files = {
    'asset_type' : context.get('asset_type.save_file'),
    'vetting' : context.get('vetting.save_file'),
    'token_issuer' : token_issuer_context.get('save_file'),
}

for k, f in contract_files.items() :
    ip_display.display(ip_display.JSON(root=k, filename=os.path.join(bindings.expand('${save}'), f)))

### Contract Context

In [ ]:
%%skip True
# ip_display.display(ip_display.JSON(data=context.context, root='context'))
ip_display.display(context.context)